In [1]:
import pandas as pd
import camelot

## PDF Crawler

In [2]:
# 2020/05/17 TwBartenderQuiz
def pdfextrat(url,page="http://www.fsvs.ks.edu.tw/ezfiles/1/1001/img/58/206002.pdf"):
    tables=camelot.read_pdf(url, pages=str(page),flavor="stream")
    print("Analysis Alchole PDF.....")
    print(tables[0])
    print(tables[0].parsing_report)
    table_df = tables[0].df
    return table_df

In [3]:
pdfextrat("http://www.fsvs.ks.edu.tw/ezfiles/1/1001/img/58/206002.pdf","34")

Analysis Alchole PDF.....
<Table shape=(48, 10)>
{'accuracy': 90.85, 'whitespace': 72.71, 'order': 1, 'page': 34}


,0,1,2,3,4,5,6,7,8,9
0,,,飲料調製乙級技術士技能檢定術科測試試題 A 組配方,,,,,,,
1,組別 A1,,,,,,,,,
2,,飲料名稱,成 \n 份,調,製,法,裝飾物,杯,器,皿
3,題序,,,,,,,,,
4,,,,,,,Garnish,,,
5,,Drink name,Ingredients,,Method,,,,Glassware,
6,,DC01,30ml Espresso Coffee,,義式咖啡機,,心型奶泡圖案,寬口咖啡杯,,
7,,Decorated Coffee,義式咖啡(7g),Pour,,,,,,
8,,咖啡拉花,Top Up with Foaming Milk,,注入法,,,,,
9,,,加滿奶泡,,,,,,,


In [4]:
def table_transform(table_df):
    rule = ["題序","一","二","三","四","五"]
    index, team = [], ""
    for num,i in enumerate(table_df.iloc[:,0]):
        if "組別" in i:
            team=i.replace("組別 ","")
        if i in rule:
            index.append(num)
            
    index[0]=index[0]+2
    
    alc_dict = {
        "Team":[],
        "Drinkname":[],
        "Ingredients":[],
        "Method":[],
        "Garnish":[],
        "Glassware":[]
    }
    
    for n,first in enumerate(index):
        dr,ing,me,gar,glass="","","","",""
        second=0
        if first == index[len(index)-1]:
            second = len(table_df)-1
        else:
            second = index[n+1]
        for i in range(first, second):
            if len(table_df.columns)==10:
                dr += table_df.iloc[i,1]
                ing += table_df.iloc[i,2]
                me += table_df.iloc[i,3]
                gar += table_df.iloc[i,6]            
                glass += table_df.iloc[i,7] + table_df.iloc[i,8] + table_df.iloc[i,9]
            elif len(table_df.columns)==9:
                dr += table_df.iloc[i,1]
                ing += table_df.iloc[i,2]
                me += table_df.iloc[i,3]
                gar += table_df.iloc[i,5]            
                glass += table_df.iloc[i,6] + table_df.iloc[i,7] + table_df.iloc[i,8]
            elif len(table_df.columns)==8:
                dr += table_df.iloc[i,1]
                ing += table_df.iloc[i,2]
                me += table_df.iloc[i,3]
                gar += table_df.iloc[i,4]            
                glass += table_df.iloc[i,5] + table_df.iloc[i,6] + table_df.iloc[i,7]
            elif len(table_df.columns)==7:
                dr += table_df.iloc[i,1]
                ing += table_df.iloc[i,2]
                me += table_df.iloc[i,4]
                gar += table_df.iloc[i,5]            
                glass += table_df.iloc[i,6]
            else:
                dr += table_df.iloc[i,1]
                ing += table_df.iloc[i,2]
                me += table_df.iloc[i,3]
                gar += table_df.iloc[i,4]            
                glass += table_df.iloc[i,5]
        alc_dict["Team"].append(team)
        alc_dict["Drinkname"].append(dr)
        alc_dict["Ingredients"].append(ing)
        alc_dict["Method"].append(me)
        alc_dict["Garnish"].append(gar)
        alc_dict["Glassware"].append(glass)
  
    alc_df = pd.DataFrame(alc_dict)
    return alc_df    

In [5]:
# 34-47 test

In [6]:
def multiplepage(page_start,page_end,url="http://www.fsvs.ks.edu.tw/ezfiles/1/1001/img/58/206002.pdf"):
    alldf=pd.DataFrame()
    for i in range(page_start,page_end+1):
        table_df=pdfextrat(url,i)
        try:
            df1=table_transform(table_df)
            alldf=pd.concat([alldf,df1])
            alldf.index=range(0,len(alldf))
        except:
            print("Error: can not transform this page {}, maybe it is not formula page".format(i))
    return alldf

In [7]:
alldf=multiplepage(34,47)

Analysis Alchole PDF.....
<Table shape=(48, 10)>
{'accuracy': 90.85, 'whitespace': 72.71, 'order': 1, 'page': 34}
Analysis Alchole PDF.....
<Table shape=(60, 6)>
{'accuracy': 99.51, 'whitespace': 64.72, 'order': 1, 'page': 35}
Analysis Alchole PDF.....
<Table shape=(53, 8)>
{'accuracy': 95.98, 'whitespace': 71.46, 'order': 1, 'page': 36}
Analysis Alchole PDF.....
<Table shape=(14, 6)>
{'accuracy': 99.62, 'whitespace': 54.76, 'order': 1, 'page': 37}
Analysis Alchole PDF.....
<Table shape=(56, 7)>
{'accuracy': 99.37, 'whitespace': 68.62, 'order': 1, 'page': 38}
Analysis Alchole PDF.....
<Table shape=(38, 10)>
{'accuracy': 92.56, 'whitespace': 76.32, 'order': 1, 'page': 39}
Analysis Alchole PDF.....
<Table shape=(23, 8)>
{'accuracy': 100.0, 'whitespace': 66.3, 'order': 1, 'page': 40}
Error: can not transform this page 40, maybe it is not formula page
Analysis Alchole PDF.....
<Table shape=(55, 9)>
{'accuracy': 96.0, 'whitespace': 75.15, 'order': 1, 'page': 41}
Analysis Alchole PDF.....
<T

In [8]:
alldf.head()

,Team,Drinkname,Ingredients,Method,Garnish,Glassware
0,A1,Drink nameDC01Decorated Coffee咖啡拉花,Ingredients30ml Espresso Coffee義式咖啡(7g)Top Up ...,Pour,心型奶泡圖案,Glassware寬口咖啡杯
1,A1,CS01EspressoDaiquiri義式戴吉利,30ml White Rum白蘭姆酒30ml Cold Espresso Coffee冰義式...,Shake,Float ThreeCoffee Beans三粒咖啡豆,Cocktail Glass雞尾酒杯
2,A1,CST01Manhattan曼哈頓(製作三杯),15ml Sugar Syrup糖水50ml Rye or Canadian Whisky裸...,Stir,Cherry櫻桃,MartiniGlass馬丁尼杯
3,A1,CBU01Hot Toddy熱托地,Dash Angostura Bitters少許安格式苦精45ml Brandy白蘭地15m...,Build,Lemon Slice檸檬片CinnamonPowder,Toddy Glass托地杯
4,A1,CS02AbsintheFrappe艾碧斯芙萊蓓,15ml Sugar Syrup糖水Top Up with Boiling Water熱開水...,Shake,肉桂粉Mint Spring薄荷枝,Old FashionedGlass古典酒杯


## Import postgreSQL

In [9]:
import sqlalchemy
from sqlalchemy import Table, Column, ForeignKey, Integer, String, VARCHAR
import psycopg2

In [10]:
def to_postgresql(inputdata, user, password, host, port, db):
    try:
        url = 'postgresql://{}:{}@{}:{}/{}'
        url = url.format(user, password, host, port, db)
        con = sqlalchemy.create_engine(url, client_encoding='utf8')
        meta = sqlalchemy.MetaData(bind=con, reflect=True)
        quiz = Table('BartenderQuiz', meta,  
                      Column('Team', VARCHAR(10)),  
                      Column('Drinkname', String),
                      Column('Ingredients', String),
                      Column('Method', String),
                      Column('Garnish', String),
                      Column('Glassware', String)
                      )
        meta.create_all(con)
        for i in range(0, len(inputdata)):
            row = quiz.insert().values(Team=inputdata.iloc[i,0], Drinkname=inputdata.iloc[i,1],
                                       Ingredients=inputdata.iloc[i,2], Method=inputdata.iloc[i,3],
                                       Garnish=inputdata.iloc[i,4], Glassware=inputdata.iloc[i,5])
            con.execute(row)            
        return "Success"
    except:
        return "Error: Mabye input wrong"

In [11]:
to_postgresql(alldf, "postgres", "root", "localhost", 5432, "alchole")

c:\users\administrator\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:6: SADeprecationWarning: The MetaData.reflect flag is deprecated and will be removed in a future release.   Please use the MetaData.reflect() method.
  


'Success'